In [135]:
import numpy as np
import pandas as pd
import string
import re
from collections import defaultdict

In [136]:
import glob

path = r'/content/drive/My Drive/Instagram_Data' 
fileNames = glob.glob(path + "/*.csv")

lst = []

for fileName in fileNames:
    print(fileName)
    df = pd.read_csv(fileName, index_col=None, header=0)
    lst.append(df)

df = pd.concat(lst, axis=0, ignore_index=True)

/content/drive/My Drive/Instagram_Data/mitpics.csv
/content/drive/My Drive/Instagram_Data/yoga.csv
/content/drive/My Drive/Instagram_Data/world.csv
/content/drive/My Drive/Instagram_Data/viceindia.csv
/content/drive/My Drive/Instagram_Data/abcwnn.csv
/content/drive/My Drive/Instagram_Data/aamaadmiparty.csv
/content/drive/My Drive/Instagram_Data/talentmoves.csv
/content/drive/My Drive/Instagram_Data/ageofempires.csv
/content/drive/My Drive/Instagram_Data/ageofempiresgame.csv
/content/drive/My Drive/Instagram_Data/voxdotcom.csv
/content/drive/My Drive/Instagram_Data/oxford_uni.csv
/content/drive/My Drive/Instagram_Data/nature_africa.csv
/content/drive/My Drive/Instagram_Data/life.csv
/content/drive/My Drive/Instagram_Data/pbsnature.csv
/content/drive/My Drive/Instagram_Data/healthyfoodvideos.csv
/content/drive/My Drive/Instagram_Data/nature.research.csv
/content/drive/My Drive/Instagram_Data/eucouncil.csv
/content/drive/My Drive/Instagram_Data/hidden_shots_.csv
/content/drive/My Drive/In

In [137]:
df.head()

,Unnamed: 0,ID,shortcode,text,hashtags,comments,likes
0,1,NaN,NaN,👏 One of MIT's newest Nobel laureates Esther D...,mit womeninscience womeninstem nobelprize nobe...,|Congratulations!!🙌🙌🙌|What was the award for?|...,3785.0
1,2,NaN,NaN,Congratulations to MIT’s Abhijit Banerjee and ...,mit nobelprize nobellaureate economicsciences ...,|Congratulations!|congratulations🎉🎉🎉🎉|@jarad_p...,4645.0
2,3,NaN,NaN,Last Friday mechanical engineering seniors tak...,thisismit mit productengineering teambuilding ...,"|🔥🔥🔥🔥🔥🔥🔥🔥|Is ""cracker causeway"" kind of a give...",2919.0
3,4,NaN,NaN,Nearly 10 years ago Priyanka Bakaya MBA ’11 fo...,mit sustainability entrepreneurs plasticintofu...,|enerjide çevreci adim atmak geleceğimizi güçl...,2860.0
4,5,NaN,NaN,At MIT.nano we think every day is Nano Day. Bu...,mit nanotechnology nanotecnologia nanoscience ...,|Smart dust|@takkksh|@photechno|@jarad_ponce|@...,5555.0


In [138]:
print(len(df))

16253


In [139]:
print(df["text"].iloc[1])

Congratulations to MIT’s Abhijit Banerjee and Esther Duflo have won the Nobel Prize in economics “for their experimental approach to alleviating global poverty." They share the prize with Michael Kremer of Harvard.

Reposted from @nobelprize_org (@get_regrann) 
The research conducted by this year’s Laureates has considerably improved our ability to fight global poverty. In just two decades their new experiment-based approach has transformed development economics which is now a flourishing field of research. 
Despite recent dramatic improvements one of humanity’s most urgent issues is the reduction of global poverty in all its forms. More than 700 million people still subsist on extremely low incomes. Every year around five million children under the age of five still die of diseases that could often have been prevented or cured with inexpensive treatments. Half of the world’s children still leave school without basic literacy and numeracy skills. 
This year’s Laureates have introduced 

In [140]:
print(df["hashtags"].iloc[1])

mit nobelprize nobellaureate economicsciences economics poverty endpoverty research experiment science society prize award nobelprize2019 nobelprizeannouncement economicsprize globalpoverty education health healthcare childhealth development regrann


In [141]:
print(df["comments"].iloc[1])

|Congratulations!|congratulations🎉🎉🎉🎉|@jarad_ponce|Congratulations|😍😍😍🔥🔥|Congratulations 🎊 🎊 🎊 🎊|🇮🇳|Great!! Congratulations to the winners and to the MIT|Congratulations from South Africa👏|The reason for I love MIT 🔥🔥🔥 🇮🇳💪💗|Congratulations! 🙏|🎉🙏😁|🙏🏻🙌🏼|There is no such thing as a nobel prize in economics|@e.lauer|Congratulations from India|MIT MITMITMIT MIT ❤️❤️❤️❤️|शुभकामनाएं|❤️|Amazing!!! Congratulations 😍|Many congratulations..|👏👏👏👏👏👏👏👏|❤️|excellent🙌🙌🙌|Congrats 👏👏👏|@cuartosectoreco|Enhorabuena!!|Really glad to have such eminent personal in MIT N now I know why MIT is best university 😀|Congratulations!


## Data Processing

### Cleaning Data

In [142]:
#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',str(text))

In [143]:
# appending first 25 comments into one big comment
def appendComments(comment):
  lst = comment.split("|")
  final_comment = ""
  for idx,entry in enumerate(lst):
    if idx == 25:
      break
    entry.strip()
    final_comment +=  entry + " "
  return final_comment

In [144]:
def cleanComment(comment):
  deEmojified_comment = deEmojify(comment)
  return appendComments(deEmojified_comment)

In [145]:
df["comments"] = df["comments"].apply(lambda x:cleanComment(x))

In [146]:
#combining comments and actual post
df["text_with_comments"] = df["text"]+ " "+ df["comments"]

In [147]:
# removing punctuations,and converting upper to lower case letters

punctuations = string.punctuation
table = punctuations.maketrans(punctuations+string.ascii_uppercase,
                               " "*len(punctuations)+string.ascii_lowercase,)

def cleanPosts(sentence):
  sentence.strip()
  sen = sentence.translate(table)
  return sen

def cleanHashTags(sentence):
  sentence.strip()
  sen = sentence.translate(table)
  return "$start " + sen + " end$"


In [148]:
post_df = df["text_with_comments"].apply(lambda w:cleanPosts(str(w)))
hashtag_df = df["hashtags"].apply(lambda w: cleanHashTags(str(w)))

In [149]:
print(hashtag_df.iloc[1])

$start mit nobelprize nobellaureate economicsciences economics poverty endpoverty research experiment science society prize award nobelprize2019 nobelprizeannouncement economicsprize globalpoverty education health healthcare childhealth development regrann end$


#### Hashtags and Likes

In [150]:
# here we only consider hashtags that are appearing atleast 10 times
def count_hashtag_likes(all_posts_hashtags,all_posts_likes):
  hashtag_likesCount = defaultdict(int)
  hashtagAppearanceCount = defaultdict(int)
  hashtag_first10_likesCount = defaultdict(int)
  for hashtags,count in zip(all_posts_hashtags,all_posts_likes):
    hashtags = str(hashtags).split()
    for hashtag in hashtags:
      hashtagAppearanceCount[hashtag] += 1
      if hashtagAppearanceCount[hashtag] >= 10:
        if hashtag not in hashtag_likesCount:
          hashtag_likesCount[hashtag] = hashtag_first10_likesCount[hashtag] + count
        hashtag_likesCount[hashtag] += count
        continue
      hashtag_first10_likesCount[hashtag] += count      
  return hashtag_likesCount      

In [151]:
hashtag_likesCount = count_hashtag_likes(hashtag_df.values.tolist(),df["likes"].values.tolist())
print(len(hashtag_likesCount))

2038


In [152]:
# list of hashtags appearing atleast 10 times
frequent_hashtags = hashtag_likesCount.keys()
print(len(frequent_hashtags))

2038


#### Splitting Data to Train ,Test and Validation Data

In [153]:
from sklearn.model_selection import train_test_split
X, X_val, Y, Y_val = train_test_split(post_df,hashtag_df, test_size=0.10,shuffle = True)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.10,shuffle = True)

In [154]:
print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)
print(X_test.shape,Y_test.shape)

(13164,) (13164,)
(1626,) (1626,)
(1463,) (1463,)


In [155]:
type(X_test)

pandas.core.series.Series

### Creating vector representations

In [156]:
class ConstructIndexes():
  def __init__(self, data):
    
    self.data = data
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()

    self.create_index()
        
  def create_index(self):
    # we consider words appearing atleas 10 times
    wordAppearanceCount = defaultdict(int)
    for phrase in self.data:
      for word in phrase.split():
        wordAppearanceCount[word] += 1
        if wordAppearanceCount[word] >= 10:
          self.vocab.add(word)
            
    # sort the vocab
    self.vocab = sorted(self.vocab)

    # add a padding token with index 0
    self.word2idx['<pad>'] = 0
        
    # word to index mapping
    for index, word in enumerate(self.vocab):
        self.word2idx[word] = index + 1 # +1 because of pad token
        
    # index to word mapping
    for word, index in self.word2idx.items():
        self.idx2word[index] = word      

In [157]:
# creating word2idx, idx2word on train data
# we use utils constructed on train to convert test and validation data
post_utils = ConstructIndexes(X_train.values.tolist())
hashtag_utils = ConstructIndexes(Y_train.values.tolist())

In [158]:
print(len(post_utils.word2idx))
print(len(hashtag_utils.word2idx))

13617
1697


In [159]:
# creating numeric vector corresponding to each post+comment
def toNumVec(constructIndexClass,df):
  tensor = []
  for phrase in df:
    phrase_lst = []
    for word in phrase.split():
      if word in constructIndexClass.word2idx:
        phrase_lst.append(constructIndexClass.word2idx[word])
    tensor.append(phrase_lst)
  return tensor

In [160]:
# Converting X_train,Y_train text sentences into numeric vectors
input_tensor_train = toNumVec(post_utils,X_train)
target_tensor_train = toNumVec(hashtag_utils,Y_train)
print(len(input_tensor_train),len(target_tensor_train))

13164 13164


In [161]:
# Converting X_val,Y_val text sentences into numeric vectors
input_tensor_val = toNumVec(post_utils,X_val)
target_tensor_val = toNumVec(hashtag_utils,Y_val)
print(len(input_tensor_val),len(target_tensor_val))

1626 1626


In [162]:
# Converting X_test,Y_test text sentences into numeric vectors
input_tensor_test = toNumVec(post_utils,X_test)
target_tensor_test = toNumVec(hashtag_utils,Y_test)
print(len(input_tensor_test),len(target_tensor_test))

1463 1463


In [163]:
print(input_tensor_train[1])
print(target_tensor_train[1])

[5811, 6986, 12654, 11610, 10471, 8621, 12310, 5504, 4786, 11715, 10256, 719, 8963, 776, 6487, 8763, 13412, 4571, 12146, 5460, 11715, 4890, 4345, 13412, 6408, 8757, 7694, 2888, 2890, 5962, 8759, 8763, 8767, 5849, 5052, 11543, 9973, 337, 1357, 1997, 11161, 337, 3005, 6988, 6120, 11161, 337, 3002, 10471, 11543, 9973, 11161, 337, 3002, 8787, 11161, 337, 3002, 10471, 5274, 5845, 3121, 4604, 5849, 11785, 7003, 6120, 12763, 11610, 6957, 719, 8007, 1997, 6957, 10695, 3002, 11543, 9973, 337, 1357, 2387, 10695, 6988, 11610, 2387, 12547, 337, 3552, 1357, 8748, 1357, 2387, 11574, 6988, 6120, 1357, 10471, 3002, 1166, 1166, 1166, 3002, 6889, 4922, 6170, 5488, 10695, 1357, 5274, 5845, 3121, 13398, 10414, 6957, 6835, 337, 9115, 11161, 337, 3002, 5845, 3121, 10471, 8007, 10471, 6988, 6120, 10695, 3002, 776, 1357, 2387, 6994]
[1, 697, 1010, 314, 316, 787, 1144, 1147, 1150, 747, 643, 467]


#### Padding Numeric Vectors to same size

In [164]:
max_length_inp = max([len(t) for t in input_tensor_train])
max_length_tar = max([len(t) for t in target_tensor_train])
print(max_length_inp,max_length_tar)

1275 42


In [165]:
def pad_sequences(x, max_len):
  padded_x = np.zeros((max_len), dtype=np.int64)
  if len(x) > max_len:
    padded_x = x[:max_len]
  else:
    padded_x[:len(x)] = x
  return padded_x

In [166]:
# padding zeros at the end of train tensors
input_tensor_train = np.array([pad_sequences(x, max_length_inp) for x in input_tensor_train if len(x) > 0])
target_tensor_train = np.array([pad_sequences(x, max_length_tar) for x in target_tensor_train if len(x) > 0])

In [167]:
# padding zeros at the end of validation tensors
input_tensor_val = np.array([pad_sequences(x, max_length_inp) for x in input_tensor_val if len(x) > 0])
target_tensor_val = np.array([pad_sequences(x, max_length_tar) for x in target_tensor_val if len(x) > 0])

In [168]:
# padding zeros at the end of test tensors
input_tensor_test= np.array([pad_sequences(x, max_length_inp) for x in input_tensor_test if len(x) > 0])
target_tensor_test = np.array([pad_sequences(x, max_length_tar) for x in target_tensor_test if len(x) > 0])

### Import Pytorch Libraries

In [169]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time

print(torch.__version__)
from torch.utils.data import Dataset, DataLoader

1.6.0+cu101


#### DataLoader for batch inputs

In [170]:
# conver the data to tensors and pass to the Dataloader 
# to create an batch iterator

class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

## Parameters
Let's define the hyperparameters and other things we need for training our NMT model.

In [171]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 64
units = 128
vocab_inp_size = len(post_utils.word2idx)
vocab_tar_size = len(hashtag_utils.word2idx)

train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)

dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

### Encoder Model

In [172]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens, device):
        # x: batch_size, max_length 
        
        # x: batch_size, max_length, embedding_dim
        x = self.embedding(x) 
                
        # x transformed = max_len X batch_size X embedding_dim
        # x = x.permute(1,0,2)
        x = pack_padded_sequence(x, lens) # unpad
    
        self.hidden = self.initialize_hidden_state(device)
        
        # output: max_length, batch_size, enc_units
        # self.hidden: 1, batch_size, enc_units
        output, self.hidden = self.gru(x, self.hidden) # gru returns hidden state of all timesteps as well as hidden state at last timestep
        
        # pad the sequence to the max length in the batch
        output, _ = pad_packed_sequence(output)
        
        return output, self.hidden

    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_sz, self.enc_units)).to(device)

In [173]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

### Testing the Encoder
Before proceeding with training, we should always try to test out model behavior such as the size of outputs just to make that things are going as expected. In PyTorch this can be done easily since everything comes in eager execution by default.

In [174]:
### Testing Encoder part
# TODO: put whether GPU is available or not
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)

print(enc_output.size()) # max_length, batch_size, enc_units

torch.Size([510, 64, 128])


### Decoder Model

In [175]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim + self.enc_units, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        # used for attention
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
    
    def forward(self, x, hidden, enc_output):
        # enc_output original: (max_length, batch_size, enc_units)
        # enc_output converted == (batch_size, max_length, hidden_size)
        enc_output = enc_output.permute(1,0,2)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        # score: (batch_size, max_length, hidden_size) # Bahdanaus's
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        # It doesn't matter which FC we pick for each of the inputs
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        #score = torch.tanh(self.W2(hidden_with_time_axis) + self.W1(enc_output))
          
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        # takes case of the right portion of the model above (illustrated in red)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # ? Looks like attention vector in diagram of source
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        
        # passing the concatenated vector to the GRU
        # output: (batch_size, 1, hidden_size)
        output, state = self.gru(x)
        
        
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((1, self.batch_sz, self.dec_units))

#### Testing the Decoder

In [176]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

print("Input: ", x.shape)
print("Output: ", y.shape)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)
print("Encoder Output: ", enc_output.shape) # batch_size X max_length X enc_units
print("Encoder Hidden: ", enc_hidden.shape) # batch_size X enc_units (corresponds to the last state)

decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)
decoder = decoder.to(device)

#print(enc_hidden.squeeze(0).shape)

dec_hidden = enc_hidden#.squeeze(0)
dec_input = torch.tensor([[hashtag_utils.word2idx['$start']]] * BATCH_SIZE)
print("Decoder Input: ", dec_input.shape)
print("--------")

for t in range(1, y.size(1)):
    # enc_hidden: 1, batch_size, enc_units
    # output: max_length, batch_size, enc_units
    predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
    
    print("Prediction: ", predictions.shape)
    print("Decoder Hidden: ", dec_hidden.shape)
    
    #loss += loss_function(y[:, t].to(device), predictions.to(device))
    
    dec_input = y[:, t].unsqueeze(1)
    print(dec_input.shape)
    break

Input:  torch.Size([64, 1275])
Output:  torch.Size([64, 42])
Encoder Output:  torch.Size([775, 64, 128])
Encoder Hidden:  torch.Size([1, 64, 128])
Decoder Input:  torch.Size([64, 1])
--------
Prediction:  torch.Size([64, 1697])
Decoder Hidden:  torch.Size([1, 64, 128])
torch.Size([64, 1])


## Training Model

In [177]:
def train(epochs,encoder,decoder,device,ytrain_utils,
          loss_function,batch_size,optimizer,dataLoader):

  for epoch in range(epochs):
      start = time.time()
      total_loss = 0

      encoder.train()
      decoder.train()
      
      for (batch, (inp, targ, inp_len)) in enumerate(dataLoader):
          
          loss = 0

          xs, ys, lens = sort_batch(inp, targ, inp_len)
          
          enc_output, enc_hidden = encoder(xs.to(device), lens, device)
          dec_hidden = enc_hidden
          
          # use teacher forcing - feeding the target as the next input (via dec_input)
          dec_input = torch.tensor([[ytrain_utils.word2idx['$start']]] * batch_size)
          
          # run code below for every timestep in the ys batch
          for t in range(1, ys.size(1)):
              predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                          dec_hidden.to(device), 
                                          enc_output.to(device))
              loss += loss_function(ys[:, t].to(device), predictions.to(device))
              #loss += loss_
              dec_input = ys[:, t].unsqueeze(1)
              
          
          batch_loss = (loss / int(ys.size(1)))
          total_loss += batch_loss
          
          optimizer.zero_grad()
          
          loss.backward()

          optimizer.step()
          
          '''if batch % 100 == 0:
              print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                          batch,
                                                          batch_loss.detach().item()))'''
          
          
      ### TODO: Save checkpoint for model
      print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                          total_loss / N_BATCH))
      print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    

In [178]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
    mask = real.ge(1).type(torch.cuda.FloatTensor)
    
    loss_ = criterion(pred, real) * mask 
    return torch.mean(loss_)

In [179]:

BATCH_SIZE = 64
## TODO: Combine the encoder and decoder into one class
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=0.001)

In [180]:
epochs = 50
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train(epochs,encoder,decoder,device,hashtag_utils,
      loss_function,BATCH_SIZE,optimizer,dataset)

Epoch 1 Loss 0.5687
Time taken for 1 epoch 72.29287242889404 sec

Epoch 2 Loss 0.4400
Time taken for 1 epoch 72.55029320716858 sec

Epoch 3 Loss 0.3965
Time taken for 1 epoch 72.8018729686737 sec

Epoch 4 Loss 0.3660
Time taken for 1 epoch 73.026859998703 sec

Epoch 5 Loss 0.3426
Time taken for 1 epoch 72.92470335960388 sec

Epoch 6 Loss 0.3248
Time taken for 1 epoch 72.8529109954834 sec

Epoch 7 Loss 0.3104
Time taken for 1 epoch 73.24533677101135 sec

Epoch 8 Loss 0.2971
Time taken for 1 epoch 73.49484252929688 sec

Epoch 9 Loss 0.2851
Time taken for 1 epoch 73.4104552268982 sec

Epoch 10 Loss 0.2744
Time taken for 1 epoch 72.5112555027008 sec

Epoch 11 Loss 0.2631
Time taken for 1 epoch 72.75166869163513 sec

Epoch 12 Loss 0.2531
Time taken for 1 epoch 72.49494504928589 sec

Epoch 13 Loss 0.2430
Time taken for 1 epoch 73.1577959060669 sec

Epoch 14 Loss 0.2331
Time taken for 1 epoch 72.13152861595154 sec

Epoch 15 Loss 0.2236
Time taken for 1 epoch 73.03208780288696 sec

Epoch 16 Lo

### Predict hashtags

In [181]:
encoder_pred = Encoder(vocab_inp_size, embedding_dim, units, 1)
decoder_pred = Decoder(vocab_tar_size, embedding_dim, units, units,1)

encoder_pred.to(device)
decoder_pred.to(device)


Decoder(
  (embedding): Embedding(1697, 64)
  (gru): GRU(192, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1697, bias=True)
  (W1): Linear(in_features=128, out_features=128, bias=True)
  (W2): Linear(in_features=128, out_features=128, bias=True)
  (V): Linear(in_features=128, out_features=1, bias=True)
)

#### Loading weights into another model

In [182]:
encoder_pred.load_state_dict(encoder.state_dict())
decoder_pred.load_state_dict(decoder.state_dict())

<All keys matched successfully>

In [183]:
softmax = nn.Softmax(dim=1)

def predict(X_test):
  results = []
  encoder_pred.eval()
  decoder_pred.eval()

  for xs in enumerate(X_test):
    xs = xs[1]
    length = np.sum(1 - np.equal(xs, 0))
    xs = torch.tensor(xs).view(-1,1)
    length = torch.tensor(length).view(-1,)

    enc_output, enc_hidden = encoder_pred(xs.to(device), length, device)
    dec_hidden = enc_hidden
        
    dec_input = torch.tensor([[hashtag_utils.word2idx['$start']]])
    curr_hashtags = []
    # run code below till we generate "end$" tag or 10 hashtags
    for t in range(1,10):
      predictions, dec_hidden, _ = decoder_pred(dec_input.to(device), 
                                          dec_hidden.to(device), 
                                          enc_output.to(device))
      #print(predictions.size())
      top_val,top_idx  = softmax(predictions).topk(1,dim = 1)
      #print(top_idx.item())
      if top_idx == hashtag_utils.word2idx["end$"]:
        break
      pred_hashtag = hashtag_utils.idx2word[top_idx.item()]
      curr_hashtags.append(pred_hashtag)
      dec_input = torch.tensor([top_idx]).unsqueeze(1)

    results.append(curr_hashtags)

  return results

In [184]:
#predict hashtags  on test data
predicted_hashtags = predict(input_tensor_test)

In [185]:
for i in range(10):
  print(Y_test.iloc[i],predicted_hashtags[i])

$start art illustration racism history end$ ['mythbusters', 'starwars']
$start naturepbs end$ ['metoo', 'timesup']
$start folkmusic pop experimentalmusic music indianmusic gujarat end$ ['commonwealth', 'commonwealthsecretariat', 'connectedcommonwealth', 'ourcommonwealth', 'chogm2018', 'chogm18', 'chogm', 'chogm2018', 'commonwealth']
$start politics mueller muellernews end$ ['science']
$start atalbiharivajpayee narendramodi merrychristmas christmas modi india photography end$ ['merrychristmas', 'happyholidays', 'selfie', 'photooftheday', 'science', 'sciencechannel']
$start tbt wlaspen wlsnowmass end$ ['cover', 'talent', 'voice', 'vocals', 'wow', 'singing', 'singer', 'karaoke', 'song']
$start notredame notredameparis notredamecathedral fire fires flame blaze spire burning tragedy firefighter firefighters firemen firefighterlife cathedral cathedrale monument monuments landmark church history paris parisfrance france french france français end$ ['fire', 'tragedy', 'church', 'learning', 'di